# Implied Volatility: A Teaching Notebook

**Learning Objectives:**
1. Understand **Monotonicity**: Why higher volatility always means higher option prices.
2. Explore **Sensitivity**: How volatility impacts ITM, ATM, and OTM options differently.
3. Verify **Consistency**: Confirm we can round-trip $\sigma \to \text{Price} \to \sigma_{\text{implied}}$.

We rely on `qpl` (Quant Pricing Lab) for analytic pricing and solving.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
import os

# Ensure we can import qpl from src if not installed
sys.path.insert(0, os.path.abspath("../src"))

from qpl.engines.analytic.black_scholes import bs_price, implied_volatility
from qpl.instruments.options import EuropeanOption
from qpl.market.curves import FlatRateCurve, FlatDividendCurve
from qpl.market.market import Market
from qpl.models.black_scholes import BlackScholesModel

# Plot styling
plt.style.use('ggplot')
print("Imports successful.")


## 1. Monotonicity: Price vs. Volatility

Option prices (calls and puts) are strictly increasing with respect to volatility (Vega > 0). 
Let's modify $\sigma$ while keeping Spot, Strike, and Time constant.


In [ ]:
# Parameters
S, K, T = 100.0, 100.0, 1.0
r, q = 0.05, 0.01

# Grid of sigmas
sigmas = np.linspace(0.05, 1.0, 50)
prices_call = []
prices_put = []

for sig in sigmas:
    p_call = bs_price(S=S, K=K, T=T, r=r, q=q, sigma=sig, kind='call')
    p_put  = bs_price(S=S, K=K, T=T, r=r, q=q, sigma=sig, kind='put')
    prices_call.append(p_call)
    prices_put.append(p_put)

# Plot
plt.figure(figsize=(10, 5))
plt.plot(sigmas, prices_call, label='Call Price')
plt.plot(sigmas, prices_put, label='Put Price')
plt.xlabel('Volatility ($\sigma$)')
plt.ylabel('Option Price')
plt.title(f'Price vs Volatility (S={S}, K={K})')
plt.legend()
plt.grid(True)
plt.show()


## 2. Sensitivity: ATM vs ITM vs OTM

Vega (sensitivity to volatility) is highest for At-The-Money (ATM) options and decays as options move deep In-The-Money (ITM) or Out-Of-The-Money (OTM).


In [ ]:
strikes = [80, 100, 120]  # ITM, ATM, OTM (for a Call with S=100)
labels = ['ITM (K=80)', 'ATM (K=100)', 'OTM (K=120)']

plt.figure(figsize=(10, 5))

for K_val, lbl in zip(strikes, labels):
    prices = []
    for sig in sigmas:
        p = bs_price(S=100, K=K_val, T=1.0, r=0.05, q=0.01, sigma=sig, kind='call')
        prices.append(p)
    plt.plot(sigmas, prices, label=lbl)

plt.xlabel('Volatility ($\sigma$)')
plt.ylabel('Call Price')
plt.title('Volatility Sensitivity by Moneyness')
plt.legend()
plt.grid(True)
plt.show()


## 3. Repricing Consistency

We can verify our solver by choosing a target $\sigma$, computing the Black-Scholes price, and ignoring $\sigma$ to see if `implied_volatility()` recovers it.


In [ ]:
# Test Cases
test_cases = [
    # Kind, Strike, Sigma_True
    ('call', 100, 0.20),
    ('call', 100, 0.50),
    ('put',  100, 0.20),
    ('put',   90, 0.30),   # OTM Put
    ('call', 110, 0.15),   # OTM Call
]

results = []
S, T, r, q = 100.0, 1.0, 0.05, 0.01
market = Market(S, FlatRateCurve(r), FlatDividendCurve(q))

print(f"Checking consistency (S={S}, T={T})...")

for kind, K, sig_true in test_cases:
    # 1. Forward Price
    price = float(bs_price(S=S, K=K, T=T, r=r, q=q, sigma=sig_true, kind=kind))
    
    # 2. Invert
    option = EuropeanOption(kind=kind, strike=K, expiry=T)
    sig_implied = implied_volatility(price, option, market)
    
    # 3. Record
    results.append({
        'Kind': kind,
        'Strike': K,
        'Sigma_True': sig_true,
        'BS_Price': price,
        'Sigma_Implied': sig_implied,
        'Error': sig_implied - sig_true
    })

df = pd.DataFrame(results)
display(df) # Jupyter pretty print
